In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import base64
import time

# ==================================================
# 1️⃣ Función auxiliar para decodificar textos ocultos
# ==================================================
def decode_text(tag):
    """Devuelve texto visible o decodifica si está en data-auth (Base64)"""
    if tag is None:
        return ""
    if tag.has_attr("data-auth"):
        try:
            return base64.b64decode(tag["data-auth"]).decode("utf-8")
        except Exception:
            return tag.get_text(strip=True)
    return tag.get_text(strip=True)

# ==================================================
# 2️⃣ Obtener lista de carreras (desde la página principal)
# ==================================================
base = "https://admision.unmsm.edu.pe/Website20261/A/"
index_url = base + "A.html"

resp = requests.get(index_url)
soup = BeautifulSoup(resp.text, "html.parser")

escuelas = []
for a in soup.select("a[href$='results.html']"):
    nombre = a.text.strip()
    link = base + a["href"]
    escuelas.append((nombre, link))

print(f"📚 Carreras encontradas: {len(escuelas)}\n")

# ==================================================
# 3️⃣ Recorrer todas las carreras y extraer resultados
# ==================================================
todos = []

for nombre, link in escuelas:
    print(f"⏳ Procesando: {nombre}")
    try:
        r = requests.get(link, timeout=10)
        s = BeautifulSoup(r.text, "html.parser")
        tabla = s.find("table", {"id": "tablaPostulantes"})
        if not tabla:
            print(f"⚠ No se encontró tabla en {nombre}")
            continue

        filas = []
        for tr in tabla.find_all("tr"):
            cols = tr.find_all("td")
            if not cols:
                continue
            codigo = cols[0].get_text(strip=True)
            apellidos_nombres = decode_text(cols[1].find("span"))
            escuela = decode_text(cols[2].find("span"))
            puntaje = cols[3].get("data-score") or cols[3].get_text(strip=True)
            merito = cols[4].get("data-merit") or cols[4].get_text(strip=True)
            observ = cols[5].get_text(strip=True)
            filas.append([codigo, apellidos_nombres, escuela, puntaje, merito, observ])

        df = pd.DataFrame(
            filas,
            columns=["Código","Apellidos_y_Nombres","Escuela","Puntaje","Mérito_E.P","Observación"]
        )
        df["Fuente"] = link
        todos.append(df)
        print(f"✔ {nombre}: {len(df)} registros")
        time.sleep(0.5)
    except Exception as e:
        print(f"❌ Error en {nombre}: {e}")

# ==================================================
# 4️⃣ Combinar y exportar todo
# ==================================================
if todos:
    final = pd.concat(todos, ignore_index=True)
    final = final.fillna("")
    final.to_csv("Resultados_UNMSM_2026I_TODAS.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ CSV final generado con {len(final)} registros totales.")
else:
    print("❌ No se pudo obtener información.")


📚 Carreras encontradas: 95

⏳ Procesando: ADMINISTRACIÃN - HUARAL
✔ ADMINISTRACIÃN - HUARAL: 11 registros
⏳ Procesando: ADMINISTRACIÃN - LIMA
✔ ADMINISTRACIÃN - LIMA: 531 registros
⏳ Procesando: ADMINISTRACIÃN - S.J.L
✔ ADMINISTRACIÃN - S.J.L: 36 registros
⏳ Procesando: ADMINISTRACIÃN DE LA GASTRONOMÃA
✔ ADMINISTRACIÃN DE LA GASTRONOMÃA: 130 registros
⏳ Procesando: ADMINISTRACIÃN DE NEGOCIOS INTERNACIONALES - HUARAL
✔ ADMINISTRACIÃN DE NEGOCIOS INTERNACIONALES - HUARAL: 31 registros
⏳ Procesando: ADMINISTRACIÃN DE NEGOCIOS INTERNACIONALES - LIMA
✔ ADMINISTRACIÃN DE NEGOCIOS INTERNACIONALES - LIMA: 929 registros
⏳ Procesando: ADMINISTRACIÃN DE NEGOCIOS INTERNACIONALES - S.J.L
✔ ADMINISTRACIÃN DE NEGOCIOS INTERNACIONALES - S.J.L: 60 registros
⏳ Procesando: ADMINISTRACIÃN DE TURISMO - HUARAL
✔ ADMINISTRACIÃN DE TURISMO - HUARAL: 18 registros
⏳ Procesando: ADMINISTRACIÃN DE TURISMO - LIMA
✔ ADMINISTRACIÃN DE TURISMO - LIMA: 162 registros
⏳ Procesando: ADMINISTRACIÃN DE

In [11]:
# =========================================================
# DESCARGA COMPLETA RESULTADOS UNMSM 2025-II
# =========================================================
import os
import pandas as pd
from bs4 import BeautifulSoup

# =============================
# CONFIGURACIÓN
# =============================
INDEX_PATH = r"C:\Users\alejo\OneDrive - Universidad Nacional Mayor de San Marcos\Resultafos\2025-II\ADMISION 2025-II\ADMISION 2025-II\admision.unmsm.edu.pe\Website20252General\index.html"
BASE_DIR = os.path.dirname(INDEX_PATH)
ROOT_DIR = os.getcwd()

# =============================
# FUNCIONES
# =============================
def leer_html(path):
    """Abre un archivo HTML local y devuelve el objeto BeautifulSoup."""
    try:
        with open(path, "r", encoding="utf-8") as f:
            return BeautifulSoup(f, "html.parser")
    except Exception as e:
        print(f"⚠️ No se pudo leer {path}: {e}")
        return None

def extraer_tabla(path):
    """Extrae la tabla principal de una página HTML si existe."""
    soup = leer_html(path)
    if not soup:
        return pd.DataFrame()
    table = soup.find("table")
    if table:
        try:
            df = pd.read_html(str(table))[0]
            # Filtrar filas que contienen “Volver al índice”
            df = df[~df.astype(str).apply(lambda x: x.str.contains("Volver", case=False)).any(axis=1)]
            return df
        except Exception as e:
            print(f"⚠️ Error al leer tabla de {path}: {e}")
    return pd.DataFrame()

# =============================
# PROCESAR MODALIDADES Y ESCUELAS
# =============================
print(f"📂 Leyendo índice desde: {INDEX_PATH}")
soup_index = leer_html(INDEX_PATH)
if not soup_index:
    raise SystemExit("❌ No se encontró el archivo del índice principal.")

modalidades = soup_index.select("table a")
resultados = []

for mod in modalidades:
    mod_nombre = mod.text.strip()
    mod_href = mod.get("href")
    mod_path = os.path.join(BASE_DIR, mod_href)
    print(f"\n➡ Modalidad: {mod_nombre}")

    soup_mod = leer_html(mod_path)
    if not soup_mod:
        continue

    escuelas = soup_mod.select("table a")
    for esc in escuelas:
        esc_nombre = esc.text.strip()
        esc_href = esc.get("href")
        esc_path = os.path.join(BASE_DIR, esc_href.replace("./", "").replace("/", os.sep))

        if not os.path.exists(esc_path):
            print(f"   ⚠️ Archivo no encontrado: {esc_path}")
            continue

        df = extraer_tabla(esc_path)
        if not df.empty:
            df["Modalidad"] = mod_nombre
            df["Escuela"] = esc_nombre
            resultados.append(df)
            print(f"   ✔ {esc_nombre}: {len(df)} registros extraídos")
        else:
            print(f"   ⚠️ Sin datos en {esc_nombre}")

# =============================
# EXPORTAR CSV EN LA CARPETA DEL SCRIPT
# =============================
if resultados:
    final = pd.concat(resultados, ignore_index=True)
    # Normalizar nombres de columnas si existen
    final.columns = [c.replace(" ", "_").replace(".", "").strip() for c in final.columns]
    output_file = os.path.join(ROOT_DIR, "Resultados_Admision_2025II.csv")
    final.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ Extracción completada: {len(final)} registros totales")
    print(f"📁 Archivo generado en: {output_file}")
else:
    print("❌ No se extrajeron resultados.")


📂 Leyendo índice desde: C:\Users\alejo\OneDrive - Universidad Nacional Mayor de San Marcos\Resultafos\2025-II\ADMISION 2025-II\ADMISION 2025-II\admision.unmsm.edu.pe\Website20252General\index.html

➡ Modalidad: EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁSICA ALTERNATIVA (EBA)
   ✔ Volver al índice: 6 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ DERECHO: 2171 registros extraídos
   ✔ DERECHO - VILLA RICA: 46 registros extraídos
   ✔ CIENCIA POLÍTICA: 326 registros extraídos
   ✔ LITERATURA: 108 registros extraídos
   ✔ LENGUAS, TRADUCCIÓN E INTERPRETACIÓN: 255 registros extraídos
   ✔ FILOSOFÍA: 39 registros extraídos
   ✔ LINGUÍSTICA: 65 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ COMUNICACIÓN SOCIAL: 374 registros extraídos
   ✔ ARTE: 57 registros extraídos
   ✔ BIBLIOTECOLOGÍA Y CIENCIAS DE LA INFORMACIÓN: 36 registros extraídos
   ✔ DANZA: 6 registros extraídos
   ✔ CONSERVACIÓN Y RESTAURACIÓN: 41 registros extraídos
   ✔ EDUCACIÓN INICIAL: 150 registros extraídos
   ✔ EDUCACIÓN PRIMARIA: 130 registros extraídos
   ✔ EDUCACIÓN SECUNDARIA: 349 registros extraídos


...

C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ INGENIERÍA DE SISTEMAS: 33 registros extraídos
   ✔ INGENIERÍA DE SOFTWARE: 23 registros extraídos
   ✔ CIENCIA DE LA COMPUTACIÓN: 6 registros extraídos

➡ Modalidad: VICTIMAS DEL TERRORISMO
   ✔ Volver al índice: 6 registros extraídos
   ✔ DERECHO: 1 registros extraídos
   ✔ INGENIERÍA AGROINDUSTRIAL: 1 registros extraídos
   ✔ ADMINISTRACIÓN - LIMA: 1 registros extraídos
   ✔ ECONOMÍA: 1 registros extraídos
   ✔ INGENIERÍA DE SISTEMAS: 2 registros extraídos

➡ Modalidad: COMUNIDADES NATIVAS
   ✔ Volver al índice: 6 registros extraídos
   ✔ DERECHO: 12 registros extraídos
   ✔ LENGUAS, TRADUCCIÓN E INTERPRETACIÓN: 7 registros extraídos
   ✔ EDUCACIÓN INICIAL: 3 registros extraídos
   ✔ EDUCACIÓN SECUNDARIA: 1 registros extraídos
   ✔ EDUCACIÓN FÍSICA: 1 registros extraídos
   ✔ INGENIERÍA QUÍMICA: 1 registros extraídos
   ✔ ADMINISTRACIÓN - LIMA: 5 registros extraídos
   ✔ ADMINISTRACIÓN MARÍTIMA Y PORTUARIA: 1 registros extraídos
   ✔ GENÉTICA Y BIOTECNOLOGÍA: 3 registros extraí

C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES - LIMA: 2 registros extraídos
   ✔ ADMINISTRACIÓN DE LA GASTRONOMÍA: 2 registros extraídos
   ✔ MARKETING: 2 registros extraídos
   ✔ CONTABILIDAD - LIMA: 1 registros extraídos
   ✔ CRIMINALÍSTICA FINANCIERA FORENSE: 2 registros extraídos
   ✔ ECONOMÍA: 3 registros extraídos
   ✔ ECONOMÍA PÚBLICA: 1 registros extraídos
   ✔ MATEMÁTICA: 1 registros extraídos
   ✔ HISTORIA: 1 registros extraídos
   ✔ SOCIOLOGÍA: 1 registros extraídos
   ✔ TRABAJO SOCIAL: 1 registros extraídos
   ✔ INGENIERÍA AMBIENTAL: 1 registros extraídos
   ✔ ARQUITECTURA Y URBANISMO: 5 registros extraídos
   ✔ INGENIERÍA INDUSTRIAL: 2 registros extraídos
   ✔ INGENIERÍA TEXTIL Y CONFECCIONES: 1 registros extraídos
   ✔ INGENIERÍA ELÉCTRICA - CHILCA: 1 registros extraídos
   ✔ INGENIERÍA DE TELECOMUNICACIONES: 1 registros extraídos
   ✔ INGENIERÍA BIOMÉDICA: 3 registros extraídos
   ✔ INGENIERÍA DE SISTEMAS: 7 registros extraídos
   ✔ INGENIERÍA DE SOFTWARE: 1 registros 

C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2485132176.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

In [ ]:
import os, time, re, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# =============================
# CONFIGURACIÓN DEL DRIVER
# =============================
CHROMEDRIVER_PATH = "C:/chromedriver-win64/chromedriver.exe"

options = Options()
options.add_argument("--headless=new")  # ejecuta sin abrir ventana
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# =============================
# URL BASE
# =============================
BASE_ROOT = "https://admision.unmsm.edu.pe/Website20251"
INDEX_URL = f"{BASE_ROOT}/A.html"
MODALIDAD = "EDUCACIÓN BÁSICA REGULAR Y ALTERNATIVA (EBR-EBA)"

# =============================
# FUNCIÓN: EXTRAER TABLA COMPLETA DE UNA CARRERA
# =============================
def extraer_tabla_completa(driver, url, carrera):
    driver.get(url)

    # esperar la tabla
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "tablaPostulantes"))
    )
    time.sleep(1)

    # mostrar 100 registros por página
    try:
        select = driver.find_element(By.NAME, "tablaPostulantes_length")
        select.send_keys("100")
        time.sleep(1.5)
    except:
        pass

    # obtener total esperado
    total_txt = driver.find_element(By.ID, "tablaPostulantes_info").text
    total = int(re.search(r"de un total de (\d[\d,\.]*)", total_txt).group(1).replace(",", "").replace(".", ""))
    print(f"→ {carrera}: {total} registros esperados")

    registros = []
    pagina = 1

    while True:
        # esperar que cargue tabla
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#tablaPostulantes tbody tr"))
        )

        # parsear
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tabla = soup.find("table", id="tablaPostulantes")
        for tr in tabla.select("tbody tr"):
            celdas = [td.get_text(strip=True) for td in tr.find_all("td")]
            if celdas:
                registros.append(celdas)

        print(f"   Página {pagina}: {len(registros)} acumulados")

        # ir a siguiente
        try:
            next_btn = driver.find_element(By.ID, "tablaPostulantes_next")
            if "disabled" in next_btn.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", next_btn)
            driver.execute_script("arguments[0].click();", next_btn)
            pagina += 1
            time.sleep(1.3)
        except Exception as e:
            print(f"   ⚠ Error al pasar página: {e}")
            break

        if len(registros) >= total:
            break

    # construir DataFrame
    if registros:
        cols = ["Código", "Apellidos_y_Nombres", "Escuela", "Puntaje", "Mérito_E.P", "Observación"]
        df = pd.DataFrame(registros, columns=cols)
        df["Carrera"] = carrera
        df["Modalidad"] = MODALIDAD
        df["Periodo"] = "2025-I"
        df["Fuente"] = url
        print(f"   ✔ {carrera}: {len(df)} registros totales extraídos.\n")
        return df
    else:
        print(f"   ⚠ Sin datos para {carrera}")
        return pd.DataFrame()

# =============================
# RECORRER TODAS LAS CARRERAS
# =============================
driver.get(INDEX_URL)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, "html.parser")

escuelas = [
    (a.text.strip(), f"{BASE_ROOT}/{a['href'].replace('./','')}")
    for a in soup.select("a[href$='0.html']")
]

print(f"\n🧭 Modalidad A: {len(escuelas)} carreras encontradas\n")

dfs = []
for nombre, link in escuelas:
    try:
        df = extraer_tabla_completa(driver, link, nombre)
        if not df.empty:
            dfs.append(df)
    except Exception as e:
        print(f"   ❌ Error en {nombre}: {e}")
    time.sleep(0.5)

driver.quit()

# =============================
# GUARDAR CSV FINAL
# =============================
os.makedirs("Resultados_2025I", exist_ok=True)
if dfs:
    final = pd.concat(dfs, ignore_index=True)
    salida = os.path.join("Resultados_2025I", "Resultados_UNMSM_2025I_A.csv")
    final.to_csv(salida, index=False, encoding="utf-8-sig")
    print(f"\n✅ Archivo guardado: {salida} ({len(final)} registros)")
else:
    print("❌ No se obtuvieron datos.")




🧭 Modalidad A: 91 carreras encontradas

→ MEDICINA HUMANA: 5065 registros esperados
   Página 1: 100 acumulados
   Página 2: 200 acumulados
   Página 3: 300 acumulados
   Página 4: 400 acumulados
   Página 5: 500 acumulados
   Página 6: 600 acumulados
   Página 7: 700 acumulados
   Página 8: 800 acumulados
   Página 9: 900 acumulados
   Página 10: 1000 acumulados
   Página 11: 1100 acumulados
   Página 12: 1200 acumulados
   Página 13: 1300 acumulados
   Página 14: 1400 acumulados
   Página 15: 1500 acumulados
   Página 16: 1600 acumulados
   Página 17: 1700 acumulados
   Página 18: 1800 acumulados
   Página 19: 1900 acumulados
   Página 20: 2000 acumulados
   Página 21: 2100 acumulados
   Página 22: 2200 acumulados
   Página 23: 2300 acumulados
   Página 24: 2400 acumulados
   Página 25: 2500 acumulados
   Página 26: 2600 acumulados
   Página 27: 2700 acumulados
   Página 28: 2800 acumulados
   Página 29: 2900 acumulados
   Página 30: 3000 acumulados
   Página 31: 3100 acumulados
   

In [ ]:
import os
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# =============================
# CONFIGURACIÓN DEL DRIVER
# =============================
CHROMEDRIVER_PATH = r"C:\chromedriver-win64\chromedriver.exe"
BASE_URL = "https://admision.unmsm.edu.pe/Website20242/"
INDEX_URL = BASE_URL + "index.html"

options = Options()
# ⚠️ Quita el modo headless para evitar errores de carga
# options.add_argument("--headless=new")
options.add_argument("--start-maximized")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# =============================
# FUNCIÓN PARA EXTRAER TABLAS
# =============================
def extraer_tabla(url):
    """Extrae tabla de resultados de una carrera."""
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table")
        if not table:
            return pd.DataFrame()
        df = pd.read_html(str(table))[0]
        return df
    except Exception as e:
        print(f"⚠️ Error al procesar {url}: {e}")
        return pd.DataFrame()

# =============================
# NAVEGAR POR MODALIDADES
# =============================
print(f"🔍 Cargando modalidades desde: {INDEX_URL}")
driver.set_page_load_timeout(60)  # Tiempo extra de espera
driver.get(INDEX_URL)
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
modalidades = soup.select("table a")

if not modalidades:
    print("❌ No se encontraron modalidades. Verifica la URL.")
    driver.quit()
    exit()

resultados = []

for mod in modalidades:
    mod_nombre = mod.text.strip()
    mod_href = mod.get("href")
    if not mod_href:
        continue

    mod_url = BASE_URL + mod_href
    print(f"\n➡ Procesando modalidad: {mod_nombre}")
    driver.get(mod_url)
    time.sleep(2)

    soup_mod = BeautifulSoup(driver.page_source, "html.parser")
    escuelas = soup_mod.select("table a")

    for esc in escuelas:
        esc_nombre = esc.text.strip()
        esc_href = esc.get("href")
        if not esc_href:
            continue

        esc_url = BASE_URL + esc_href.replace("./", "")
        print(f"   → Extrayendo {esc_nombre}")

        df = extraer_tabla(esc_url)
        if not df.empty:
            df["Modalidad"] = mod_nombre
            df["Escuela"] = esc_nombre
            resultados.append(df)
            print(f"      ✔ {len(df)} registros extraídos")
        else:
            print(f"      ⚠️ Sin datos en {esc_nombre}")
        time.sleep(1.5)

# =============================
# EXPORTAR CSV FINAL
# =============================
driver.quit()
if resultados:
    final_df = pd.concat(resultados, ignore_index=True)
    output_file = os.path.join(os.getcwd(), "Resultados_Admision_2024II.csv")
    final_df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ Extracción completa: {len(final_df)} registros totales")
    print(f"📁 Archivo guardado en: {output_file}")
else:
    print("❌ No se extrajeron resultados.")


📂 Leyendo índice desde: C:\Users\alejo\OneDrive - Universidad Nacional Mayor de San Marcos\Resultafos\2024-II\ADMISION 2024-II\ADMISION 2024-II\admision.unmsm.edu.pe\Website20242\index.html

➡ Modalidad: EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁSICA ALTERNATIVA (EBA)
   ✔ Volver al índice: 12 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


   ✔ MEDICINA HUMANA: 3823 registros extraídos
   ✔ OBSTETRICIA: 307 registros extraídos
   ✔ ENFERMERÍA: 452 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ TEC. MED. LAB. CLÍNICO Y ANATOMÍA PATOLÓGICA: 126 registros extraídos
   ✔ TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN: 115 registros extraídos
   ✔ TEC. MED. RADIOLOGÍA: 77 registros extraídos
   ✔ TEC. MED. TERAPIA OCUPACIONAL: 37 registros extraídos
   ✔ NUTRICION - LIMA: 187 registros extraídos
   ✔ NUTRICION - VILLA RICA: 7 registros extraídos


...

   ✔ EDUCACIÓN FÍSICA - LIMA: 3 registros extraídos
   ✔ INGENIERÍA AGROINDUSTRIAL - LIMA: 1 registros extraídos
   ✔ INGENIERÍA MECÁNICA DE FLUIDOS: 1 registros extraídos
   ✔ INGENIERÍA BIOMÉDICA: 1 registros extraídos
   ✔ INGENIERÍA DE SISTEMAS: 2 registros extraídos
   ✔ INGENIERÍA DE SOFTWARE: 1 registros extraídos

➡ Modalidad: DEPORTISTAS CALIFICADOS DE ALTO NIVEL
   ✔ Volver al índice: 12 registros extraídos
   ✔ MEDICINA HUMANA: 1 registros extraídos
   ✔ TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN: 1 registros extraídos
   ✔ NUTRICION - LIMA: 1 registros extraídos
   ✔ EDUCACIÓN FÍSICA - LIMA: 1 registros extraídos
   ✔ INGENIERÍA AGROINDUSTRIAL - LIMA: 1 registros extraídos
   ✔ INGENIERÍA DE TELECOMUNICACIONES: 1 registros extraídos

➡ Modalidad: VICTIMAS DEL TERRORISMO
   ✔ Volver al índice: 12 registros extraídos
   ✔ MEDICINA HUMANA: 3 registros extraídos
   ✔ ENFERMERÍA: 1 registros extraídos
   ✔ NUTRICION - LIMA: 1 registros extraídos
   ✔ INGENIERÍA BIOMÉDICA: 2 regis

C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ MEDICINA HUMANA: 26 registros extraídos
   ✔ TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN: 1 registros extraídos
   ✔ TEC. MED. RADIOLOGÍA: 2 registros extraídos
   ✔ DERECHO - LIMA: 12 registros extraídos
   ✔ LENGUAS, TRADUCCIÓN E INTERPRETACIÓN: 3 registros extraídos
   ✔ FILOSOFÍA: 1 registros extraídos
   ✔ COMUNICACIÓN SOCIAL: 1 registros extraídos
   ✔ ARTE: 1 registros extraídos
   ✔ FARMACIA Y BIOQUÍMICA: 1 registros extraídos
   ✔ TOXICOLOGÍA: 1 registros extraídos
   ✔ ODONTOLOGÍA: 2 registros extraídos
   ✔ EDUCACIÓN INICIAL: 1 registros extraídos
   ✔ EDUCACIÓN FÍSICA - LIMA: 1 registros extraídos
   ✔ INGENIERÍA QUÍMICA: 1 registros extraídos
   ✔ MEDICINA VETERINARIA: 2 registros extraídos
   ✔ ADMINISTRACIÓN DE LA GASTRONOMÍA: 1 registros extraídos
   ✔ MARKETING: 1 registros extraídos
   ✔ CONTABILIDAD - LIMA: 1 registros extraídos
   ✔ ECONOMÍA: 3 registros extraídos
   ✔ COMPUTACIÓN CIENTÍFICA: 1 registros extraídos
   ✔ HISTORIA: 1 registros extraídos
   ✔ ARQUEOL

C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2599070978.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   ✔ ADMINISTRACIÓN - LIMA: 9 registros extraídos
   ✔ ADMINISTRACIÓN DE TURISMO - LIMA: 2 registros extraídos
   ✔ MICROBIOLOGÍA Y PARASITOLOGÍA: 4 registros extraídos
   ✔ INGENIERÍA MECÁNICA DE FLUIDOS: 1 registros extraídos
   ✔ MATEMÁTICA: 2 registros extraídos
   ✔ COMPUTACIÓN CIENTÍFICA: 1 registros extraídos
   ✔ HISTORIA: 2 registros extraídos
   ✔ ANTROPOLOGÍA: 1 registros extraídos
   ✔ ARQUEOLOGÍA: 1 registros extraídos
   ✔ TRABAJO SOCIAL: 9 registros extraídos
   ✔ INGENIERÍA GEOLÓGICA - LIMA: 3 registros extraídos
   ✔ INGENIERÍA DE SEGURIDAD Y SALUD EN EL TRABAJO: 2 registros extraídos
   ✔ PSICOLOGÍA - LIMA: 13 registros extraídos
   ✔ PSICOLOGÍA ORGANIZACIONAL Y DE LA GESTIÓN HUMANA: 2 registros extraídos
   ✔ INGENIERÍA ELECTRÓNICA: 3 registros extraídos
   ✔ INGENIERÍA ELÉCTRICA - LIMA: 2 registros extraídos
   ✔ INGENIERÍA BIOMÉDICA: 10 registros extraídos
   ✔ INGENIERÍA DE SISTEMAS: 6 registros extraídos
   ✔ INGENIERÍA DE SOFTWARE: 10 registros extraídos

✅ Extr

In [8]:
import os
import pandas as pd
from bs4 import BeautifulSoup

# =============================
# CONFIGURACIÓN
# =============================
INDEX_PATH = r"C:\Users\alejo\OneDrive - Universidad Nacional Mayor de San Marcos\Resultafos\2024-I\ADMISION 2024-I\ADMISION 2024-I\admision.unmsm.edu.pe\Website20241\index.html"
BASE_DIR = os.path.dirname(INDEX_PATH)

# Carpeta raíz = ubicación del script actual
ROOT_DIR = os.getcwd()

# =============================
# FUNCIONES
# =============================
def leer_html(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return BeautifulSoup(f, "html.parser")
    except Exception as e:
        print(f"⚠️ No se pudo leer {path}: {e}")
        return None

def extraer_tabla(path):
    soup = leer_html(path)
    if not soup:
        return pd.DataFrame()
    table = soup.find("table")
    if table:
        try:
            df = pd.read_html(str(table))[0]
            return df
        except Exception as e:
            print(f"⚠️ Error al leer tabla de {path}: {e}")
    return pd.DataFrame()

# =============================
# PROCESAR MODALIDADES Y ESCUELAS
# =============================
print(f"📂 Leyendo índice desde: {INDEX_PATH}")
soup_index = leer_html(INDEX_PATH)
if not soup_index:
    raise SystemExit("❌ No se encontró el archivo del índice principal.")

modalidades = soup_index.select("table a")
resultados = []

for mod in modalidades:
    mod_nombre = mod.text.strip()
    mod_href = mod.get("href")
    mod_path = os.path.join(BASE_DIR, mod_href)
    print(f"\n➡ Modalidad: {mod_nombre}")

    soup_mod = leer_html(mod_path)
    if not soup_mod:
        continue

    escuelas = soup_mod.select("table a")
    for esc in escuelas:
        esc_nombre = esc.text.strip()
        esc_href = esc.get("href")
        esc_path = os.path.join(BASE_DIR, esc_href.replace("./", "").replace("/", os.sep))

        if not os.path.exists(esc_path):
            print(f"   ⚠️ Archivo no encontrado: {esc_path}")
            continue

        df = extraer_tabla(esc_path)
        if not df.empty:
            df["Modalidad"] = mod_nombre
            df["Escuela"] = esc_nombre
            resultados.append(df)
            print(f"   ✔ {esc_nombre}: {len(df)} registros extraídos")
        else:
            print(f"   ⚠️ Sin datos en {esc_nombre}")

# =============================
# EXPORTAR CSV EN LA CARPETA DEL SCRIPT
# =============================
if resultados:
    final = pd.concat(resultados, ignore_index=True)
    output_file = os.path.join(ROOT_DIR, "Resultados_Admision_2024I.csv")
    final.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ Extracción completada: {len(final)} registros totales")
    print(f"📁 Archivo generado en: {output_file}")
else:
    print("❌ No se extrajeron resultados.")


📂 Leyendo índice desde: C:\Users\alejo\OneDrive - Universidad Nacional Mayor de San Marcos\Resultafos\2024-I\ADMISION 2024-I\ADMISION 2024-I\admision.unmsm.edu.pe\Website20241\index.html

➡ Modalidad: EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁSICA ALTERNATIVA (EBA)
   ✔ Volver al índice: 1 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


   ✔ MEDICINA HUMANA: 5383 registros extraídos
   ✔ OBSTETRICIA: 414 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


   ✔ ENFERMERÍA: 548 registros extraídos
   ✔ TEC. MED. LAB. CLÍNICO Y ANATOMÍA PATOLÓGICA: 140 registros extraídos
   ✔ TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN: 152 registros extraídos
   ✔ TEC. MED. RADIOLOGÍA: 98 registros extraídos
   ✔ TEC. MED. TERAPIA OCUPACIONAL: 63 registros extraídos


...

C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


   ✔ INGENIERÍA DE TELECOMUNICACIONES: 205 registros extraídos
   ✔ INGENIERÍA BIOMÉDICA: 303 registros extraídos


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


   ✔ INGENIERÍA DE SISTEMAS: 1776 registros extraídos
   ✔ INGENIERÍA DE SOFTWARE: 832 registros extraídos
   ✔ CIENCIAS DE LA COMPUTACIÓN: 60 registros extraídos

✅ Extracción completada: 28863 registros totales
📁 Archivo generado en: c:\Users\alejo\Documents\GitHub\PrediccionIngresoUNMSM\Resultados_Admision_2024I.csv


C:\Users\alejo\AppData\Local\Temp\ipykernel_17628\2217156865.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
